In [11]:
from lxml import etree
import sqlite3

In [12]:
xml_db = 'wordnet/data/xml_synsets.db'
entry_db = 'wordnet/data/xml_entries.db'

In [13]:
tree = etree.parse("estwn-et-2.0.0.beta.xml")

In [15]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def synsetTable(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS xml_synsets(
                                        
                                        id INT NOT NULL,
                                        sourceSense TEXT NOT NULL,
                                        pos TEXT NOT NULL,
                                        sense INT NOT NULL,
                                        writtenForm TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")  
        

In [24]:
def upload_data(entry_xml_db,sset_xml_db):
    #TODO: replace "_" with " " in sourceSense.
    synsetTable(sset_xml_db)
    conn = create_connection(sset_xml_db)
    cursor = conn.cursor()

    with conn:
        conn2 = create_connection(entry_xml_db)
        cursor2 = conn2.cursor()
        with conn2:
            i=0
            for estId in estwnId_list:
                cursor2.execute("SELECT pos,sense,writtenForm FROM xml_LexicalEntry WHERE estwn_id = ?", (estId,))
                row = cursor2.fetchall()
                    
                if len(row) > 1:
                    if sset_word[i] == 'None':
                        try:
                            var = row[0][2]
                            sourceSense = var.replace("_"," ")
                        except:
                            sourceSense = row[0][2]
                    else:
                        sourceSense = sset_word[i]
                    for word in row:
                        cursor.execute("INSERT INTO xml_synsets(id, sourceSense, pos, sense, writtenForm) VALUES (?,?,?,?,?)"\
                                                                                  ,(i,sourceSense, word[0],word[1],word[2]))
                    conn.commit()
                else:
                    if sset_word[i] == 'None':
                        try:
                            var = row[0][2]
                            sourceSense = var.replace("_"," ")
                        except:
                            sourceSense = row[0][2]
                    else:
                        sourceSense = sset_word[i]
                    cursor.execute("INSERT INTO xml_synsets(id, sourceSense, pos, sense, writtenForm) VALUES (?,?,?,?,?)"\
                                                                                  ,(i,sourceSense,row[0][0],row[0][1],row[0][2]))
                    conn.commit()
                i+=1

In [20]:

def fetch_estwnId(db_file):

    synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
    j=0
    for sset in synsets:
        estwnId_list.append(sset.attrib['id'])
        try:
            var   = sset[0].attrib['sourceSense']
            sset_word.append(var[2:-3])

        except:
            sset_word.append('None')

In [22]:
estwnId_list = []
sset_word = []

fetch_estwnId(xml_db)

In [25]:

upload_data(entry_db, xml_db)

In [11]:
'''
 start_estwn = sset.attrib['target']   
                sset_word   = sset.attrib['sourceSense']
                relation    = sset.attrib['relation']
                conf        = sset.attrib['confidenceScore']
                status      = sset.attrib['status']
                '''
        #TODO: else statement        
        #else:
         #   try:
                
                '''
                #print("try @", j)
                word = sset[0].attrib['writtenForm']
                if " " in word:
                    var = word.replace(" ", "_")
                    word = var
                pos  = sset[0].attrib['partOfSpeech']
                string = "s-" + word + "-" + pos
                sset_list.append(string)
                estwnId_list.append(sset.attrib['id'])
            except:
                #print("pass @",j)
                pass
                
        j+=1
        '''
        '''
        for relations
        
        if len(sset) > 2:
            for data in sset:
                #print(data.attrib)
                
                status_list.append(data.attrib['target'])
                conf_list.append((data.attrib['confidenceScore']))
                source_sense.append(data[0].attrib['sourceSense'])           
                '''
'''

IndentationError: unexpected indent (<ipython-input-11-95a00368ac9d>, line 12)